# Converting Graphs Between Types

In [17]:
import ndio
print ndio.version

0.0.20


ndio can tap the [m2g](http://m2g.io) web endpoints to convert graphs of any size between a variety of graph file fomats, including GraphML, edgelist, etc.

To begin, import the m2g `Remote`:

In [18]:
from ndio.remote.m2g import *

To get a list of all supported graph types, use the `m2g.GraphFormats` enumerable:

In [19]:
print dir(GraphFormats)

['EDGELIST', 'GRAPHDB', 'GRAPHML', 'LGL', 'MAT', 'NCOL', 'NUMPY', 'PAJEK', '__doc__', '__module__', '_any', 'edgelist', 'graphdb', 'graphml', 'lgl', 'mat', 'ncol', 'numpy', 'pajek']


Let's convert a graphml file to edgelist. We can do both simultaneously! To do this, we first instantiate a new m2g Remote object:

In [20]:
m = m2g()

By default, an m2g Remote has the default origin (openconnecto.me) and protocol (http) set. You can verify this using the `repr` function:

In [21]:
repr(m)

"ndio.remote.m2g('openconnecto.me', 'http', '')"

The last (empty) quotes refer to the default notification email. m2g notifies you by email when your requests are completed. You can either specify your email in each function call, or you can use `set_default_email` to set it once and forget it. All subsequent calls will use that email unless specified explicitly.

In [22]:
m.set_default_email('ndio-demos@neurodata.io')

Now we can go ahead and run the conversion. A graphml file, `test.graphml`, is supplied in `data/`.

## A Note About Background Processes
ndio supports threading, which means that you can run these calls in the background if you expect them to take a long time. Each call to the server offers an optional `use_threads` argument, which defaults to `False`. If set to `True`, another optional argument, `callback`, may be specified. `callback` should be a function that takes one string as its sole argument. When the server call completes, `callback(value)` is called, where `value` is a link to the completed server job's results.

A simple `callback` might be as easy as:

```python
def print_link(value):
    print value
```

No callback has to be specified, but... it seems a bit silly to ask the server to compute something and then ignore the value, no?

Perhaps a better `callback` would be the following:

```python
def download_from_link(link):
    from subprocess import call
    call(["wget", "-r", "--no-parent", link])
```

This one will use the shell's `wget` command to download every file in the returned directory. Your sole argument is a string of an http link. So any sort of function will do!

**You also have the option of running the calls synchronously,** like regular function calls. This makes more sense for small files, and so it's what we'll demonstrate here.

In [23]:
response = m.convert_graph('../data/test.graphml', GraphFormats.GRAPHML, [GraphFormats.EDGELIST])

We can follow the link to get our completed files.

In [24]:
print response

http://openconnecto.me/mrdata/tmp/formUploadSun27Dec2015_00.17.39/converted/


If we wanted to run that same graph asynchronously, which is dumb because it's so small, but this is informative:

In [25]:
def cb(value):
    print value

m.convert_graph('../data/test.graphml', GraphFormats.GRAPHML,
                [GraphFormats.EDGELIST, GraphFormats.GRAPHDB],
                use_threads=True, callback=cb)

<small>Note that we won't see the output from that in a jupyter notebook because it only reads output on the main thread.</small>